In [ ]:
from pydrake.all import (
    AddDefaultVisualization,
    DiscreteContactApproximation,
    ModelVisualizer,
    RobotDiagramBuilder,
    Simulator,
    StartMeshcat,
    namedview,
)

from underactuated import running_as_notebook
from underactuated.utils import ConfigureParser

In [ ]:
meshcat = StartMeshcat()

A simple model of the MIT Leg Laboratory's famous [Planar One-Leg Hopper](http://www.ai.mit.edu/projects/leglab/robots/2D_hopper/2D_hopper.html) and its controller.


In [ ]:
def passive_simulation():
    robot_builder = RobotDiagramBuilder(time_step=1e-3)
    parser = robot_builder.parser()
    ConfigureParser(parser)
    parser.AddModelsFromUrl(
        "package://underactuated/models/planar_one_leg_hopper.dmd.yaml"
    )
    plant = robot_builder.plant()
    plant.set_discrete_contact_approximation(DiscreteContactApproximation.kLagged)
    plant.Finalize()

    builder = robot_builder.builder()
    # meshcat.Set2dRenderMode()
    AddDefaultVisualization(builder, meshcat=meshcat)

    diagram = robot_builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    plant_context = plant.GetMyMutableContextFromRoot(context)

    PositionView = namedview(
        "Positions",
        plant.GetPositionNames(
            add_model_instance_prefix=False, always_add_suffix=False
        ),
    )
    # TODO(russt): COM of the body is at the hip, but should not be.
    positions = PositionView(plant.GetPositions(plant_context))
    positions.body_qz = 0.1
    positions.hip = -0.2
    plant.SetPositions(plant_context, positions[:])

    if running_as_notebook:
        simulator.set_target_realtime_rate(1.0)
        simulator.AdvanceTo(5.0)
    else:
        simulator.AdvanceTo(0.1)


passive_simulation()

Controller coming soon!